In [12]:
# pip install pandas openpyxl
# pip install gspread google-auth google-auth-oauthlib google-auth-httplib2

In [13]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

import pandas as pd
import io
import base64
import datetime

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage

In [14]:
# import gspread
# from google.oauth2.credentials import Credentials
# from google.oauth2 import service_account 
# from googleapiclient.discovery import build

# SERVICE_ACCOUNT_FILE = "C:/Users/aarya/Downloads/dashboard-399006-8c9b4464fd62.json"
# SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# creds = service_account.Credentials.from_service_account_file(
#         SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# SPREADSHEET_ID = '1tHTXomXz0jbXTSPmSyGtZp3DDbdWtOYJowCzOmdPHtc'
# SAMPLE_RANGE_NAME = 'Sheet1'

# service = build('sheets', 'v4', credentials=creds)
# sheet = service.spreadsheets()
# result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range = SAMPLE_RANGE_NAME).execute()
# #print(result)

# client = gspread.authorize(creds)


In [15]:
# # Update live data

# doc = client.open_by_key(SPREADSHEET_ID)
# worksheet = doc.get_worksheet(0)  # Use the index of the worksheet (0 for the first sheet)

# # Read data from Google Sheets
# data = worksheet.get_all_values()

# # Create a DataFrame (assuming the first row contains column headers)
# df = pd.DataFrame(data[1:], columns=data[0])

In [16]:
# # print(df)
# # print(df['date of joining'].dtype)
# df['date of birth'] = pd.to_datetime(df['date of birth'], dayfirst = True)
# # print(df['date of birth'].dtype)

# df['date of joining'] = pd.to_datetime(df['date of joining'], dayfirst = True)
# # print(df['date of joining'].dtype)

In [17]:
app = dash.Dash(external_stylesheets=[dbc.themes.LUMEN])

In [18]:
df = pd.read_excel('C:/Users/aarya/Downloads/use_this_sample_excel_file (1).xlsx')
#print(df)

In [19]:
#Define function to display persons with DOB or DOJ in the current month
def display_current_month_info():
    
    current_month = get_current_month()
    birth_month_filter = df['date of birth'].dt.strftime('%m') == current_month 
    joining_month_filter = df['date of joining'].dt.strftime('%m') == current_month

   
    birth_persons = df[birth_month_filter]
    joining_persons = df[joining_month_filter]

    birth_person_info_divs = []
    joining_person_info_divs = []

    for _, row in birth_persons.iterrows():
        person_name = row['name']
        person_info_div = html.Div([
            html.H5(f'Details for {person_name}:')
        ])
        bday_person_email = row['email ID']
        bday_person_dob = row['date of birth']

        person_info_div.children.append(html.P(f'Email ID: {bday_person_email}'))
        person_info_div.children.append(html.P(f'Birth date: {bday_person_dob}'))
        # for column_name, value in row.items():
        #     if column_name != 'name':
        #         person_info_div.children.append(html.P(f'{column_name}: {value}'))
                
        birth_person_info_divs.append(person_info_div)
   
    for _, row in joining_persons.iterrows():
        person_name = row['name']
        person_info_div = html.Div([
            html.H5(f'Details for {person_name}:')
        ])

        anni_person_email = row['email ID']
        anni_person_doj = row['date of joining']

        person_info_div.children.append(html.P(f'Email ID: {anni_person_email}'))
        person_info_div.children.append(html.P(f'Work Anniversary: {anni_person_doj}'))
        # for column_name, value in row.items():
        #     if column_name != 'name':
        #         person_info_div.children.append(html.P(f'{column_name}: {value}'))

        joining_person_info_divs.append(person_info_div)

    current_month_info_div = html.Div([
        html.Div(birth_person_info_divs, className='col-md-6'),
        html.Div(joining_person_info_divs, className='col-md-6')
    ], className='row')
        
    return current_month_info_div

In [20]:
# Function to get current month
def get_current_month():
    now = datetime.datetime.now()
    return now.strftime('%m')

In [21]:
#Function to get current date
def get_current_date():
    now = datetime.datetime.now()
    return now.strftime('%d')

In [22]:
# Define the layout of the app
app.layout = html.Div(#children = 
[
    html.H1("Clevered Dashboard", className="text-center fw-bold text-decoration-underline"),
    
    html.Div(id = 'upload-data', className = 'centered-container'),
    
    html.Div([dcc.Dropdown(id='month-dropdown',options=[
                {'label': 'January', 'value': '01'},
                {'label': 'February', 'value': '02'},
                {'label': 'March', 'value': '03'},
                {'label': 'April', 'value': '04'},
                {'label': 'May', 'value': '05'},
                {'label': 'June', 'value': '06'},
                {'label': 'July', 'value': '07'},
                {'label': 'August', 'value': '08'},
                {'label': 'September', 'value': '09'},
                {'label': 'October', 'value': '10'},
                {'label': 'November', 'value': '11'},
                {'label': 'December', 'value': '12'},
            ],
            placeholder='Select a month...',
            style={'width': '90%', 'margin': '15px'},
            className="px-2 bg-white border rounded-pill"
        ),
        
        dcc.Dropdown(id='name-dropdown',placeholder='Select a person...',style={'width': '90%', 'margin': '15px'},
            className="px-2 bg-white border rounded-pill"
        ),
    ], style={'display': 'flex'}), 
    
    html.Div(id = 'search-container', className = 'centered-container', style = {'margin': '10px auto'}, 
             children = [dcc.Input(id='manual-search-input', type='text', placeholder='Enter a name...'),
                         html.Button('Search', id='manual-search-button', n_clicks=0, className = "text-center"),
                         html.Button('Clear', id = 'clear-button', n_clicks = 0, className = "text-center")]), 

    html.Div(id='output-columns'),
    

    html.Div(id='output-person-info'),
    
    html.Div([
        html.H2("Current Month Analysis", className="text-center fw-bold"),# fst-italic text-decoration-line-through"),

        html.Div([html.P('Birthdays in this month', className = "fst-italic fw-bold", style={'width': '50%', 'margin': 'auto', 'text-align': 'center', 'font-size': '20px'}),
                  html.P('Anniversaries in this month', className = "fst-italic fw-bold", style={'width': '50%', 'margin': 'auto', 'text-align': 'center', 'font-size': '20px'})], 
                  style={'display': 'flex', 'justify-content': 'center'}),
        
        html.Div(id='current-month-info', className = 'text-dark') 
    ]),
    
    display_current_month_info(),
    
    html.Div([dcc.Input(id='sender_password_input', type='password', placeholder='Enter your password...'),
              html.Button('Send Birthday Wishes', id = 'send_bday_mail', n_clicks = 0, className = 'text-center'), 
              html.Button('Send Work Anniversary Wishes', id = 'send_anni_mail', n_clicks = 0, className = 'text-center')], 
             style = {'display':'flex', 'margin': '15px'}),
    
    html.Div(id='current-month-bday-info', className = "fst-italic"),# text-decoration-line-through")
    
    html.Div(id='current-month-anni-info', className = "fst-italic")# text-decoration-line-through")

])
#  ],style={
#         'background': 'linear-gradient(360deg, #FF5733, #FFC300)',  # Replace with your desired gradient colors
#         'height': '100vh',  # Set the height to cover the entire viewport
#         'color': 'white',  # Set the text color to contrast with the background
#         'padding': '20px',  # Add padding for content within the div
#     })


In [23]:
# Function to send an email
def send_email(sender_email, password, recv_email, subject, message, image_path):
    
    try:
        # Create a message
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = recv_email
        msg['Subject'] = subject

        msg.attach(MIMEText(message, 'plain'))

        with open(image_path, 'rb') as image_file:
            image_data = image_file.read()
            image = MIMEImage(image_data)
            image.add_header('Content-Disposition', 'attachment', filename = 'happy_birthday_clevered.png')
            msg.attach(image)

        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(sender_email, password)
        #print("login successful")
        
        server.sendmail(sender_email, recv_email, msg.as_string())
        server.quit()

        return "Email sent successfully"
    except Exception as e:
        return f"Error: Unable to send email! - {str(e)}"

In [24]:
@app.callback(
    Output('current-month-bday-info', 'children'),
    Input('send_bday_mail', 'n_clicks'),
    State('sender_password_input', 'value'),
)
def send_birthday_wishes(n_clicks, password):
    if n_clicks > 0:
        sender_email = 'arya.verma.923@gmail.com'
        df['date of birth'] = pd.to_datetime(df['date of birth'])

        current_month = get_current_month()
        current_date = get_current_date()
        birth_month_filter = (df['date of birth'].dt.strftime('%m') == current_month) & (df['date of birth'].dt.strftime('%d') == current_date)
        recipients = df[birth_month_filter][['name', 'email ID']] 
        
        messages_sent = []
        for _, row in recipients.iterrows():
            recipient_name = row['name']
            recipient_email = row['email ID']
            send_email(sender_email, password, recipient_email, "Happy Birthday!", f"Happy birthday, {recipient_name}!", 'C:/Users/aarya/OneDrive/Desktop/happy_birthday_clevered.png')
            messages_sent.append(recipient_name)

        if messages_sent:
            return f"Birthday wishes successfully sent to {', '.join(messages_sent)}"
    return ''

In [25]:
@app.callback(
    Output('current-month-anni-info', 'children'),
    Input('send_anni_mail', 'n_clicks'),
    State('sender_password_input', 'value'),
)
def send_birthday_wishes(n_clicks, password):
    if n_clicks > 0:
        sender_email = 'arya.verma.923@gmail.com'
        df['date of joining'] = pd.to_datetime(df['date of joining'])

        current_month = get_current_month()
        current_date = get_current_date()
        birth_month_filter = (df['date of joining'].dt.strftime('%m') == current_month) & (df['date of joining'].dt.strftime('%d') == current_date)
        recipients = df[birth_month_filter][['name', 'email ID']] 
        
        messages_sent = []
        for _, row in recipients.iterrows():
            recipient_name = row['name']
            recipient_email = row['email ID']
            send_email(sender_email, password, recipient_email, "Happy Work Anniversary!", f"Work Anniversary, {recipient_name}!", 'C:/Users/aarya/OneDrive/Desktop/happy_birthday_clevered.png')
            messages_sent.append(recipient_name)
            
        if messages_sent:
            return f"Work anniversary successfully wishes sent to {', '.join(messages_sent)}"
    return ''


In [26]:
# Define combined callback to handle person info display, name dropdown updates, and manual search
@app.callback(
    [
        Output('output-person-info', 'children'),
        Output('name-dropdown', 'options'),
        Output('manual-search-input', 'value'),
    ],
    [
        Input('name-dropdown', 'value'),
        Input('month-dropdown', 'value'),
        Input('manual-search-button', 'n_clicks'),
        Input('clear-button', 'n_clicks'), 
        State('manual-search-input', 'value')
    ]
)
def update_person_info(selected_name, selected_month, search_clicks, clear_clicks, manual_search_name):
    
    df['date of birth'] = pd.to_datetime(df['date of birth'])#, errors='coerce').dt.date
    df['date of joining'] = pd.to_datetime(df['date of joining'])#, errors='coerce').dt.date

    output_person_info = ''
    name_dropdown_options = []

    if selected_month:
        current_month = selected_month
        birth_month_filter = df['date of birth'].dt.strftime('%m') == current_month
        joining_month_filter = df['date of joining'].dt.strftime('%m') == current_month

        filtered_persons = df[birth_month_filter | joining_month_filter]

        person_info_divs = []

        for _, row in filtered_persons.iterrows():
            person_name = row['name']
            person_info_div = html.Div([
                html.H4(f'Details for {person_name}:')
            ])

            for column_name, value in row.items():
                person_info_div.children.append(html.P(f'{column_name}: {value}'))

            person_info_divs.append(person_info_div)
            name_dropdown_options.append({'label': person_name, 'value': person_name})

    if selected_name:
        for person_info_div in person_info_divs:
            if person_info_div.children[0].children == f'Details for {selected_name}:':
                output_person_info = person_info_div
                break
    person_info = pd.DataFrame() #

    if search_clicks is not None and search_clicks > 0 and manual_search_name:
        manual_search_name_components = manual_search_name.split(' ')
        manual_search_name_first = manual_search_name_components[0].lower()
        df['name_lower'] = df['name'].str.lower()

        person_info = df[df['name_lower'].str.startswith(manual_search_name_first)].copy() 
        person_info.drop('name_lower', axis = 1, inplace = True)
        
    if not person_info.empty:
            person_info_dict = person_info.iloc[0].to_dict()
            output_person_info = html.Div([
                html.H4(f'Details for {manual_search_name}:')
            ])

            for column_name, value in person_info_dict.items():
                output_person_info.children.append(html.P(f'{column_name}: {value}'))

            name_dropdown_options.append({'label': manual_search_name, 'value': manual_search_name})

    if clear_clicks is not None and clear_clicks >0:
            manual_search_name = ''
    return output_person_info, name_dropdown_options, manual_search_name


In [27]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8060)